In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata
import pickle

In [2]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 2000)

In [3]:
data = pd.read_table('text_files/football-city-html.txt') 

In [4]:
df = data.rename(columns={"</div>": "data"})

In [5]:
terms = ['col-sm-6 col-xs-6', 'col-sm-3 hidden-xs', '/modules/images/flags/']

In [6]:
df = df[df.data.str.contains('|'.join(terms))] 

In [7]:
df.drop(index=5, inplace=True)

In [8]:
df.shape

(5782, 1)

In [9]:
df

,data
13,"<div class=""col-sm-6 col-xs-6""><b>AaB</b></div>"
18,"<img src=""/modules/images/flags/_56.png"" alt=""Denmark"" title=""Denmark"">"
20,"<div class=""col-sm-3 hidden-xs"">Aalborg</div>"
27,"<div class=""col-sm-6 col-xs-6""><b>Aalesund</b></div>"
32,"<img src=""/modules/images/flags/_140.png"" alt=""Norway"" title=""Norway"">"
...,...
27134,"<img src=""/modules/images/flags/_181.png"" alt=""Switzerland"" title=""Switzerland"">"
27136,"<div class=""col-sm-3 hidden-xs"">Bern</div>"
27143,"<div class=""col-sm-6 col-xs-6""><b>Young Buffaloes</b></div>"
27148,"<img src=""/modules/images/flags/_179.png"" alt=""Swaziland"" title=""Swaziland"">"


In [10]:
pattern = '|'.join(['<div class=\"col-sm-6 col-xs-6\"><b>', '<img src=\"/modules/images/flags/_', '<div class=\"col-sm-3 hidden-xs\">', '</b>', '</div>'])

In [11]:
df.data = df.data.str.replace(pattern, '')

/tmp/ipykernel_66044/2799521911.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.data = df.data.str.replace(pattern, '')


In [12]:
def removing_patterns(df: pd.DataFrame, col_names: list, patern: str):
    df[col_names].apply(lambda x: str(x) if re.search(pattern, str(x)) is None else re.findall(pattern, str(x)))
    return df

In [13]:
pattern = r'\"[\s|\w]+\">$'
df = removing_patterns(df, list(df), pattern)

In [14]:
def find_no_utf_symbols(sr: pd.Series):
    find_symols = r'[^.a-zA-Z0-9)(&"\s>;<=,-/}{\']'
    rare_symbols = sr.map(lambda x: ''.join(re.findall(find_symols, str(x))))
    return rare_symbols

In [15]:
rare_symbols = find_no_utf_symbols(df.data)

In [16]:
str(set(rare_symbols))

"{'', '’éé', 'ãã', 'šá', 'éé', 'łł', 'çş', 'ủầộ', 'ží', 'º', 'õ', 'ľý', 'ééá', 'ūẓī', 'éšě', 'Å', 'íí', 'ë', 'ć', 'ėž', 'éá', 'íá', 'ř', 'áá', 'ā', 'ň', 'ýŽáš', 'Łę', 'É', 'âá', 'č', 'å', 'š', 'ãç', 'áš', 'ł', 'İ', 'šć', 'ôô', 'í', 'È', 'Š', 'ď', 'ī', 'āā', 'ń', 'óę', 'â', 'ış', 'ž', 'óú', 'ø', 'ãí', 'àộ', 'šáá', 'ăăş', 'à', 'đ', 'ýáš', 'ú', 'Í', 'æ', 'ō', 'ıçğ', 'šéá', 'Ž', 'āḍ', 'ëçë', 'áí', 'è', 'ʿā', 'ññ', 'Šľ', 'ğ', 'İÖİ', 'äé', 'ľé', 'ė', 'íð', 'éḥ', 'ðø', 'Łóź', 'ó', 'áň', 'çë', 'úí', 'ä', 'čí', 'îş', 'óá', 'Ħ', 'ç', 'əə', 'üü', 'çã', 'íš', 'ı', 'ęó', 'ýŽ', 'Śś', 'çöüü', 'ū', 'óó', 'ș', 'çğ', 'áž', 'ġā', 'žá', 'Ō', 'óŁę', 'âş', 'áé', 'ăţ', 'é', 'öö', 'ã', 'čŠ', 'ľč', 'čš', 'óã', 'íáé', 'àò', 'ñ', 'ţ', 'á', 'éï', '’', 'Şı', 'žíš', 'ää', 'şş', 'əəə', 'Žž', 'ş', 'ľ', 'ěříž', 'ă', 'öð', 'ăă', 'ăăăş', 'ýí', 'šňé', 'ü', 'ööö', 'áŠ', 'ï', 'óñ', 'î', 'ážá', 'øø', 'ę', 'Şüə', 'ığ', 'šýš', 'ðæ', 'ö', 'ź', 'ą', 'éë', 'ëë', 'ā’', 'žň', 'öä', 'ô', 'éó', 'Čč', 'ê', 'áó', 'ří', 'ġāī', 'ïé', 'ý

In [17]:
all_symbols = []
for symbol in rare_symbols:
    symbols = ''.join(symbol)
    all_symbols.append(symbols)
all_symbols = str(all_symbols)
all_symbols = ''.join(set(str(all_symbols)))
all_symbols = all_symbols.replace(' ', '')
all_symbols = all_symbols.replace(']', '')
all_symbols = all_symbols.replace('[', '')
all_symbols = all_symbols.replace(',', '')
all_symbols = all_symbols.replace('\'', '')

In [18]:
all_symbols

'ºõÅəëćřāňÉčåšłẓİíÈŠďīńâžøàđČúÍæðōòŽýèğėŁóäŚçĦśủıěầūộșŌÖʿéãñţá’ġşăľḥüÑïîęöźąŞôḍê'

In [19]:
pattern_2 = r'^[0-9]+.png'
pattern_3 = r'title=\"(.+?)\"'

In [20]:
df = df.data.apply(lambda x: ''.join(re.findall(pattern_3, str(x)) if re.search(pattern_2, str(x)) is not None else x))

In [21]:
df = pd.DataFrame(df)

In [22]:
df.shape

(5782, 1)

In [23]:
df.iloc[648:649,:]

,data
3067,


In [24]:
df.data.replace('', np.nan, inplace=True)

In [25]:
df.dropna(subset=['data'], inplace=True)

In [26]:
df.shape

(5757, 1)

In [27]:
def str_decode(str_obj: str):
    normalized = unicodedata.normalize('NFD', str_obj)
    decode_str = u"".join([c for c in normalized if not unicodedata.combining(c)])
    return decode_str

In [28]:
df.data = df.data.apply(lambda x: str_decode(x))

In [29]:
rare_symbols_keys = {'ł':'l','Ł':'L','º':'','ʿ':'\'','ə':'e','Ħ':'H','đ':'d','’':'\'','ı':'i','æ':'a','ð':'d','ø':'o'}

def replace_rare_symbols(df: pd.DataFrame, col_name: str, dict_obj: dict):
    for k, v in rare_symbols_keys.items():
        df[col_name] = df[col_name].apply(lambda x: x.replace(k, v) if re.search(k, str(x)) else x)
    return df[col_name]       

In [30]:
df.data = replace_rare_symbols(df, 'data', rare_symbols_keys)

In [31]:
df.head(3)

,data
13,AaB
18,Denmark
20,Aalborg


In [32]:
df.reset_index(inplace=True)

In [33]:
df.drop('index', axis=1, inplace=True)

In [34]:
club, country, city = df[0::3], df[1::3], df[2::3]

In [35]:
table_df = pd.concat([club.reset_index(drop=True), country.reset_index(drop=True), city.reset_index(drop=True)], axis=1)

In [36]:
table_df.columns.values[0] = "club"
table_df.columns.values[1] = "country"
table_df.columns.values[2] = "city"

In [37]:
table_df.shape

(1919, 3)

In [38]:
df = table_df.copy()

In [39]:
# take city.values from hooks for transcription
df.city = df.city.map(lambda x: ''.join(re.findall(r'\((.*?)\)', x)) if re.search(r'\(', x) else x)

In [40]:
# remove Buenos Aires districts
buenos_pattern = 'Buenos'
df.city = df.city.apply(lambda x: x if re.search(buenos_pattern, str(x)) is None else 'Buenos Aires')

In [41]:
# take city.values before '/'
df.city = df.city.map(lambda x: ''.join(re.findall(r'^(.*?)/', x)) if re.search(r'\/', x) else x)

In [42]:
# take city.values before ',' without districts
df.city = df.city.map(lambda x: ''.join(re.findall(r'^(.*?),', x)) if re.search(r',', x) else x)

In [43]:
# rename 'D.F.' at 'Mexico'
mexico_pattern = 'D.F.'
df.city = df.city.apply(lambda x: x if re.search(mexico_pattern, str(x)) is None else 'Mexico')

In [44]:
df.head()

,club,country,city
0,AaB,Denmark,Aalborg
1,Aalesund,Norway,Alesund
2,Aarau,Switzerland,Aarau
3,ABC,Brazil,Natal
4,Aberdeen,Scotland,Aberdeen


In [45]:
df.tail()

,club,country,city
1914,Yokohama,Japan,Yokohama
1915,Yokohama F. Marinos,Japan,Yokohama
1916,Young Africans,Tanzania,Dar-es-Salaam
1917,Young Boys,Switzerland,Bern
1918,Young Buffaloes,Swaziland,Manzini


In [48]:
# with open('pickle_files/df_teams_data', 'wb') as f:
#     pickle.dump(df, f)

### Add teams data

In [ ]:
# with open('pickle_files/df_teams_data', 'rb') as f:
#     df_for_add = pickle.load(f)

In [ ]:
df_for_add.shape

In [ ]:
club = 'Vila Nova'
country = 'Brazil'
city = 'Goiania'

new_club = {'club':club, 'club_country':country, 'club_city':city}
new_club = pd.DataFrame([new_club])
print(new_club)
df_for_add = pd.concat([df_for_add, new_club], ignore_index=True)

In [ ]:
df_for_add.sort_values(by='club', ascending=True, inplace=True)
df_for_add.reset_index(drop=True, inplace=True)
df_for_add.shape

In [ ]:
df_for_add[df_for_add.club == club]

In [ ]:
# with open('pickle_files/df_teams_data', 'wb') as f:
#     pickle.dump(df_for_add, f)